In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# 設定自動增長 GPU 記憶體用量
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

# 設定 Keras 使用的 Session
tf.compat.v1.keras.backend.set_session(sess)

In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json
from keras.models import load_model

## Train

In [3]:
def preprocess(train_path, test_path, num):
    input_texts = []
    target_texts = []
    
    test_input_texts = []
    test_target_texts = []
    
    num_lines_train_file = sum(1 for line in open(train_path, encoding='utf-8'))
    num_lines_test_file = sum(1 for line in open(test_path, encoding='utf-8'))
                    
    print("Read",train_path,"...")
    counter = 0
    with open(train_path,  encoding='utf-8') as fp:
        for json_str in fp:
            counter = counter + 1
            data = json.loads(json_str)
            input_texts.append(data["english"])
            target_texts.append(data["chinese"])
            
            '''
            if counter%1000000==0:
                print("Now processing {}/{} rows...".format(counter, num_lines_train_file))
            '''
            if counter==num:
                break
            
    print("Read",train_path,"finished!")
    
    print("\nRead",test_path,"...")
    counter = 0
    with open(test_path,  encoding='utf-8') as fp:
        for json_str in fp:
            counter = counter + 1
            data = json.loads(json_str)
            
            if counter <= num_lines_test_file-100:
                #input_texts.append(data["english"])
                #target_texts.append(data["chinese"])
                pass
            else:
                test_input_texts.append(data["english"])
                test_target_texts.append(data["chinese"])
    print("Read",test_path,"finished!")      
          
    return input_texts, target_texts, test_input_texts, test_target_texts

In [4]:
def getInputTargetChars(input_texts, target_texts):
    print("\nProcessing chars...")
    input_characters = set()
    target_characters = set()
    for input_text, target_text in zip(input_texts, target_texts):
        target_text = '\t' + target_text + '\n'
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    print("Processing chars finished!")
    return input_characters, target_characters

In [5]:
def getEncoderDecoderData(input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data, input_token_index, target_token_index):
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            #print(char)
            encoder_input_data[i, t, input_token_index[char]] = 1.
        encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
        for t, char in enumerate(target_text):
            # decoder_target_data 领先 decoder_input_data by 一个时间步。
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                # decoder_target_data 将提前一个时间步，并且将不包含开始字符。
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.
        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
        decoder_target_data[i, t:, target_token_index[' ']] = 1.
        
    return  encoder_input_data, decoder_input_data, decoder_target_data

In [6]:
def genModel(latent_dim, num_encoder_tokens, num_decoder_tokens):
    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    
    encoder_states = [state_h, state_c]

    
    decoder_inputs = Input(shape=(None, num_decoder_tokens))

    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return model,encoder_model,decoder_model

In [7]:
def trainSaveModel(model, model_path, encoder_input_data, decoder_input_data, decoder_target_data, batch_size,epochs):
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)
    
    model.save(model_path)

In [8]:
train_path = 'translation2019zh/translation2019zh_train.json'
test_path = 'translation2019zh/translation2019zh_valid.json'
model_path = 'e2c_ep100.h5'
num_line_read = 2500

batch_size = 64 
epochs = 500
latent_dim = 256 

input_texts, target_texts, test_input_texts, test_target_texts = preprocess(train_path,test_path, num_line_read)
input_characters, target_characters = getInputTargetChars(input_texts, target_texts)
input_characters = sorted(list(set(input_characters)))
target_characters = sorted(list(set(target_characters)))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('\nNumber of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

Read translation2019zh/translation2019zh_train.json ...
Read translation2019zh/translation2019zh_train.json finished!

Read translation2019zh/translation2019zh_valid.json ...
Read translation2019zh/translation2019zh_valid.json finished!

Processing chars...
Processing chars finished!

Number of samples: 2500
Number of unique input tokens: 199
Number of unique output tokens: 3015
Max sequence length for inputs: 256
Max sequence length for outputs: 120


In [9]:
encoder_input_data, decoder_input_data, decoder_target_data = getEncoderDecoderData(input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data, input_token_index, target_token_index)

In [10]:
 model,encoder_model,decoder_model = genModel(latent_dim, num_encoder_tokens, num_decoder_tokens)

In [11]:
trainSaveModel(model, model_path, encoder_input_data, decoder_input_data, decoder_target_data, batch_size,epochs)

Epoch 1/500
32/32 [==============================] - 18s 196ms/step - loss: 2.8966 - accuracy: 0.6898 - val_loss: 2.0964 - val_accuracy: 0.7215
Epoch 2/500
32/32 [==============================] - 4s 123ms/step - loss: 2.1546 - accuracy: 0.7134 - val_loss: 2.0672 - val_accuracy: 0.7236
Epoch 3/500
32/32 [==============================] - 4s 119ms/step - loss: 2.1269 - accuracy: 0.7139 - val_loss: 2.0450 - val_accuracy: 0.7238
Epoch 4/500
32/32 [==============================] - 4s 120ms/step - loss: 2.0907 - accuracy: 0.7142 - val_loss: 2.0174 - val_accuracy: 0.7241
Epoch 5/500
32/32 [==============================] - 4s 117ms/step - loss: 2.0497 - accuracy: 0.7150 - val_loss: 1.9478 - val_accuracy: 0.7239
Epoch 6/500
32/32 [==============================] - 4s 119ms/step - loss: 1.9870 - accuracy: 0.7158 - val_loss: 2.2280 - val_accuracy: 0.7235
Epoch 7/500
32/32 [==============================] - 4s 121ms/step - loss: 1.9760 - accuracy: 0.7178 - val_loss: 1.9036 - val_accuracy: 0.727

Epoch 58/500
32/32 [==============================] - 4s 117ms/step - loss: 1.0665 - accuracy: 0.7975 - val_loss: 1.6999 - val_accuracy: 0.7545
Epoch 59/500
32/32 [==============================] - 4s 116ms/step - loss: 1.0528 - accuracy: 0.7993 - val_loss: 1.7075 - val_accuracy: 0.7539
Epoch 60/500
32/32 [==============================] - 4s 117ms/step - loss: 1.0365 - accuracy: 0.8010 - val_loss: 1.7159 - val_accuracy: 0.7536
Epoch 61/500
32/32 [==============================] - 4s 116ms/step - loss: 1.0209 - accuracy: 0.8036 - val_loss: 1.7186 - val_accuracy: 0.7537
Epoch 62/500
32/32 [==============================] - 4s 115ms/step - loss: 1.0065 - accuracy: 0.8054 - val_loss: 1.7211 - val_accuracy: 0.7540
Epoch 63/500
32/32 [==============================] - 4s 115ms/step - loss: 0.9920 - accuracy: 0.8077 - val_loss: 1.7284 - val_accuracy: 0.7544
Epoch 64/500
32/32 [==============================] - 4s 116ms/step - loss: 0.9764 - accuracy: 0.8098 - val_loss: 1.7500 - val_accuracy:

Epoch 115/500
32/32 [==============================] - 4s 119ms/step - loss: 0.4149 - accuracy: 0.9178 - val_loss: 2.1233 - val_accuracy: 0.7452
Epoch 116/500
32/32 [==============================] - 4s 117ms/step - loss: 0.4068 - accuracy: 0.9196 - val_loss: 2.1362 - val_accuracy: 0.7446
Epoch 117/500
32/32 [==============================] - 4s 115ms/step - loss: 0.3997 - accuracy: 0.9209 - val_loss: 2.1354 - val_accuracy: 0.7448
Epoch 118/500
32/32 [==============================] - 4s 117ms/step - loss: 0.3937 - accuracy: 0.9226 - val_loss: 2.1301 - val_accuracy: 0.7463
Epoch 119/500
32/32 [==============================] - 4s 116ms/step - loss: 0.3866 - accuracy: 0.9241 - val_loss: 2.1512 - val_accuracy: 0.7456
Epoch 120/500
32/32 [==============================] - 4s 116ms/step - loss: 0.3777 - accuracy: 0.9267 - val_loss: 2.1566 - val_accuracy: 0.7457
Epoch 121/500
32/32 [==============================] - 4s 118ms/step - loss: 0.3733 - accuracy: 0.9268 - val_loss: 2.1658 - val_ac

32/32 [==============================] - 4s 116ms/step - loss: 0.1399 - accuracy: 0.9785 - val_loss: 2.5925 - val_accuracy: 0.7421
Epoch 172/500
32/32 [==============================] - 4s 116ms/step - loss: 0.1342 - accuracy: 0.9799 - val_loss: 2.5956 - val_accuracy: 0.7417
Epoch 173/500
32/32 [==============================] - 4s 117ms/step - loss: 0.1359 - accuracy: 0.9792 - val_loss: 2.6101 - val_accuracy: 0.7417
Epoch 174/500
32/32 [==============================] - 4s 120ms/step - loss: 0.1283 - accuracy: 0.9810 - val_loss: 2.6152 - val_accuracy: 0.7409
Epoch 175/500
32/32 [==============================] - 4s 121ms/step - loss: 0.1271 - accuracy: 0.9812 - val_loss: 2.6415 - val_accuracy: 0.7420
Epoch 176/500
32/32 [==============================] - 4s 116ms/step - loss: 0.1247 - accuracy: 0.9814 - val_loss: 2.6362 - val_accuracy: 0.7413
Epoch 177/500
32/32 [==============================] - 4s 118ms/step - loss: 0.1237 - accuracy: 0.9815 - val_loss: 2.6456 - val_accuracy: 0.7416

32/32 [==============================] - 4s 114ms/step - loss: 0.0498 - accuracy: 0.9915 - val_loss: 2.9701 - val_accuracy: 0.7401
Epoch 228/500
32/32 [==============================] - 4s 113ms/step - loss: 0.0512 - accuracy: 0.9910 - val_loss: 2.9891 - val_accuracy: 0.7416
Epoch 229/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0497 - accuracy: 0.9912 - val_loss: 2.9983 - val_accuracy: 0.7409
Epoch 230/500
32/32 [==============================] - 4s 114ms/step - loss: 0.0483 - accuracy: 0.9913 - val_loss: 3.0198 - val_accuracy: 0.7413
Epoch 231/500
32/32 [==============================] - 4s 113ms/step - loss: 0.0498 - accuracy: 0.9907 - val_loss: 3.0270 - val_accuracy: 0.7391
Epoch 232/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0484 - accuracy: 0.9907 - val_loss: 3.0273 - val_accuracy: 0.7402
Epoch 233/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0484 - accuracy: 0.9910 - val_loss: 3.0251 - val_accuracy: 0.7399

32/32 [==============================] - 4s 116ms/step - loss: 0.0290 - accuracy: 0.9924 - val_loss: 3.2319 - val_accuracy: 0.7412
Epoch 284/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0315 - accuracy: 0.9918 - val_loss: 3.2455 - val_accuracy: 0.7403
Epoch 285/500
32/32 [==============================] - 4s 117ms/step - loss: 0.0301 - accuracy: 0.9923 - val_loss: 3.2581 - val_accuracy: 0.7408
Epoch 286/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0323 - accuracy: 0.9916 - val_loss: 3.2423 - val_accuracy: 0.7401
Epoch 287/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0321 - accuracy: 0.9917 - val_loss: 3.2390 - val_accuracy: 0.7408
Epoch 288/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0295 - accuracy: 0.9922 - val_loss: 3.2368 - val_accuracy: 0.7412
Epoch 289/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0305 - accuracy: 0.9921 - val_loss: 3.2518 - val_accuracy: 0.7403

32/32 [==============================] - 4s 115ms/step - loss: 0.0274 - accuracy: 0.9920 - val_loss: 3.3976 - val_accuracy: 0.7418
Epoch 340/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0297 - accuracy: 0.9913 - val_loss: 3.3684 - val_accuracy: 0.7413
Epoch 341/500
32/32 [==============================] - 4s 114ms/step - loss: 0.0251 - accuracy: 0.9923 - val_loss: 3.3934 - val_accuracy: 0.7414
Epoch 342/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0272 - accuracy: 0.9918 - val_loss: 3.3860 - val_accuracy: 0.7411
Epoch 343/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0263 - accuracy: 0.9922 - val_loss: 3.3758 - val_accuracy: 0.7407
Epoch 344/500
32/32 [==============================] - 4s 114ms/step - loss: 0.0246 - accuracy: 0.9924 - val_loss: 3.3880 - val_accuracy: 0.7414
Epoch 345/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0272 - accuracy: 0.9918 - val_loss: 3.3926 - val_accuracy: 0.7416

32/32 [==============================] - 4s 115ms/step - loss: 0.0250 - accuracy: 0.9921 - val_loss: 3.4743 - val_accuracy: 0.7414
Epoch 396/500
32/32 [==============================] - 4s 114ms/step - loss: 0.0246 - accuracy: 0.9921 - val_loss: 3.4701 - val_accuracy: 0.7423
Epoch 397/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0239 - accuracy: 0.9922 - val_loss: 3.4779 - val_accuracy: 0.7413
Epoch 398/500
32/32 [==============================] - 4s 114ms/step - loss: 0.0242 - accuracy: 0.9923 - val_loss: 3.4929 - val_accuracy: 0.7420
Epoch 399/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0238 - accuracy: 0.9923 - val_loss: 3.4833 - val_accuracy: 0.7418
Epoch 400/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0246 - accuracy: 0.9922 - val_loss: 3.4858 - val_accuracy: 0.7419
Epoch 401/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0240 - accuracy: 0.9923 - val_loss: 3.4606 - val_accuracy: 0.7420

32/32 [==============================] - 4s 118ms/step - loss: 0.0231 - accuracy: 0.9923 - val_loss: 3.5554 - val_accuracy: 0.7419
Epoch 452/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0236 - accuracy: 0.9922 - val_loss: 3.5531 - val_accuracy: 0.7415
Epoch 453/500
32/32 [==============================] - 4s 114ms/step - loss: 0.0236 - accuracy: 0.9923 - val_loss: 3.5390 - val_accuracy: 0.7404
Epoch 454/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0231 - accuracy: 0.9923 - val_loss: 3.5550 - val_accuracy: 0.7421
Epoch 455/500
32/32 [==============================] - 4s 116ms/step - loss: 0.0232 - accuracy: 0.9922 - val_loss: 3.5582 - val_accuracy: 0.7422
Epoch 456/500
32/32 [==============================] - 4s 115ms/step - loss: 0.0234 - accuracy: 0.9923 - val_loss: 3.5581 - val_accuracy: 0.7420
Epoch 457/500
32/32 [==============================] - 4s 119ms/step - loss: 0.0230 - accuracy: 0.9923 - val_loss: 3.5513 - val_accuracy: 0.7422

In [29]:
trainSaveModel(model, model_path, encoder_input_data, decoder_input_data, decoder_target_data, batch_size,epochs)

Epoch 1/100
50/50 [==============================] - 147s 3s/step - loss: 2.3371 - accuracy: 0.7453 - val_loss: 1.8029 - val_accuracy: 0.7609
Epoch 2/100
50/50 [==============================] - 146s 3s/step - loss: 1.7879 - accuracy: 0.7613 - val_loss: 1.7850 - val_accuracy: 0.7615
Epoch 3/100
50/50 [==============================] - 146s 3s/step - loss: 1.7583 - accuracy: 0.7620 - val_loss: 1.7372 - val_accuracy: 0.7613
Epoch 4/100
50/50 [==============================] - 153s 3s/step - loss: 1.6979 - accuracy: 0.7626 - val_loss: 1.7101 - val_accuracy: 0.7618
Epoch 5/100
50/50 [==============================] - 147s 3s/step - loss: 1.6388 - accuracy: 0.7639 - val_loss: 1.6442 - val_accuracy: 0.7670
Epoch 6/100
50/50 [==============================] - 148s 3s/step - loss: 1.6171 - accuracy: 0.7663 - val_loss: 1.6209 - val_accuracy: 0.7673
Epoch 7/100
50/50 [==============================] - 151s 3s/step - loss: 1.6030 - accuracy: 0.7677 - val_loss: 1.6140 - val_accuracy: 0.7664
Epoch 

In [30]:
trainSaveModel(model, model_path, encoder_input_data, decoder_input_data, decoder_target_data, batch_size,epochs)

Epoch 1/100
50/50 [==============================] - 155s 3s/step - loss: 0.3935 - accuracy: 0.9190 - val_loss: 1.7967 - val_accuracy: 0.7835
Epoch 2/100
50/50 [==============================] - 146s 3s/step - loss: 0.3699 - accuracy: 0.9258 - val_loss: 1.8144 - val_accuracy: 0.7832
Epoch 3/100
50/50 [==============================] - 144s 3s/step - loss: 0.3630 - accuracy: 0.9277 - val_loss: 1.8177 - val_accuracy: 0.7828
Epoch 4/100
50/50 [==============================] - 143s 3s/step - loss: 0.3574 - accuracy: 0.9285 - val_loss: 1.8294 - val_accuracy: 0.7827
Epoch 5/100
50/50 [==============================] - 148s 3s/step - loss: 0.3501 - accuracy: 0.9305 - val_loss: 1.8387 - val_accuracy: 0.7831
Epoch 6/100
50/50 [==============================] - 148s 3s/step - loss: 0.3449 - accuracy: 0.9318 - val_loss: 1.8434 - val_accuracy: 0.7826
Epoch 7/100
50/50 [==============================] - 148s 3s/step - loss: 0.3390 - accuracy: 0.9328 - val_loss: 1.8496 - val_accuracy: 0.7826
Epoch 

## Test

In [33]:
def decode_sequence(input_seq):
    # 将输入编码为状态向量。
    states_value = encoder_model.predict(input_seq)
    #print("state:",states_value)

    # 生成长度为 1 的空目标序列。
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 用起始字符填充目标序列的第一个字符。
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 一批序列的采样循环
    # (为了简化，这里我们假设一批大小为 1)。
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        #print(output_tokens)
        # 采样一个 token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 退出条件：达到最大长度或找到停止符。
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 更新目标序列（长度为 1）。
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 更新状态
        states_value = [h, c]

    return decoded_sentence

In [34]:
# 反向查询 token 索引可将序列解码回可读的内容。
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [35]:
for seq_index in range(100):
    # 抽取一个序列（训练集的一部分）进行解码。
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Correct sentence:', target_texts[seq_index])

-
Input sentence: For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液。
-
Input sentence: He calls the Green Book, his book of teachings, “the new gospel.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 他还把宣扬自己思想的所谓《绿皮书》称作“新福音书”。
-
Input sentence: And the light breeze moves me to caress her long ear
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 微风推着我去爱抚它的长耳朵
-
Input sentence: They have the blood of martyrs is the White to flow …
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                       

-
Input sentence: It shows that vertical stiffener's spaces have some effects on pure-shearing ulti…
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 而腹板鼓曲对纯弯和纯剪极限承载力的影响则可不予考虑。
-
Input sentence: "People are embarrassed to admit that's why they're giving up their pets, " said Betsy McFarland, the Humane Society's director of communications for companion animals.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: “人们羞于说出他们遗弃宠物的真正原因，”人道协会陪伴动物协调主管伊莉莎白. 麦克法兰说。
-
Input sentence: Mars gets hit in this tutorial complete with monolith from 2001 Space Odyssey all in 2D.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 火星在本教程中获得击中从2001年太空漫游所有的一块完整的二维。
-
Input sentence: Show all

-
Input sentence: From hair tip first began gradually, after all, through from downward, nodular comb.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 先从发梢开始通顺，逐渐向上，打通全部结节后，从上向下梳顺。
-
Input sentence: The sky began to be clear up a bit when we left St Gallen abbey and library.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 离开圣加仑修道院和修道院图书馆，天稍稍开始放晴。
-
Input sentence: Once more, Cinderella's fairy godmother reminded her to be home by midnight.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 仙女教母再一次提醒辛德瑞拉要在午夜前到家。
-
Input sentence: This paper introduces the demand analysis and function design in detail, gives the source codes of relevant interface functions and base a

-
Input sentence: The company is through ISO9001 quality system authentication , some products have also passed UL, CCEE authentication .
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 公司通过了ISO9001质量体系认证，部份产品还通过了UL、CCEE认证。
-
Input sentence: The Book of Revelation was also traditionally assigned to him.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 〈启示录〉传统上也认为是他的作品。
-
Input sentence: Our guest bedroom has an entire wall stacked with boxes containing unknown objects of more “stuff”.
Decoded sentence: 色列：为了我们的视来跟2福特别的 X                                                                                                       
Correct sentence: 在我们的来宾卧室里，各种装着未知东西的箱子堆满了整堵墙。
-
Input sentence: It was easily good enough for pole so that was the main thing.
Decoded sentence: 色列：为了

In [ ]:
target_token_index